## Pytorch Common Knowledge

In [3]:
import torch
from torch import nn, Tensor
import torch.nn.functional as F

## 1. view, reshape, transpose, permute 차이  

[contiguous 의미 설명 글](https://stackoverflow.com/questions/26998223/what-is-the-difference-between-contiguous-and-non-contiguous-arrays/26999092#26999092)  

C contiguous는 rows가 contiguous blocks of memory로 저장되었음을 의미한다.  
즉, 다음 memory address가 그 row의 다음 row value를 갖고 있다는 것이다.  
(column 방향은 Fortran contiguous)

In [2]:
a = torch.FloatTensor([[1,2,3], [4,5,6]])
b = torch.transpose(a, 1, 0)
# memory space 확인: a.storage().data_ptr()

In [3]:
# C contiguity is lost
print(a.is_contiguous(), b.is_contiguous())

True False


In [4]:
# view는 contiguity를 보존하지만 reshape은 그렇지 않음
a_view = a.view(3,2)
a_reshape = a.reshape(3,2)
print(a.is_contiguous())
print(b.is_contiguous())

True
False


transpose는 0, 1 사이만 바꿀 수 있지만, permute는 훨씬 자유롭게 고차원까지 커버할 수 있음  

memory efficiency를 유지하기 위해서는 reshape, transpose, permute를 사용한 후에 아래와 같이 해주는 것이 좋음

In [6]:
a_transpose = torch.transpose(a, 1, 0)
print(a_transpose.is_contiguous())
a_transpose = torch.transpose(a, 1, 0).contiguous()
print(a_transpose.is_contiguous())

False
True


## 2. register_buffer

In [4]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer(name='pe', tensor=pe, persistent=True)

    def forward(self, x: Tensor) -> Tensor:
        # x: Tensor, shape [seq_len, batch_size, embedding_dim]
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

torch.nn.Module에 register_buffer를 적용하면, 특정 Tensor나 Layer를 파라미터로 취급하지 않게 해준다.  
예를 들어 BatchNorm의 `running_mean` 은 학습 가능한 파라미터가 아니지만 Module's state의 일부이다.  

`persistent` 인자를 `False`로 하면 추후에 module의 `state_dict()` 메서드에서 등장하지 않는다.  

또한 optimizer로 업데이트하지 않고, GPU 연산이 가능하다.  

In [6]:
# self.register_buffer('running_mean', torch.zeros(num_features))
pe = PositionalEncoding(d_model=512)

In [8]:
print(pe.state_dict().keys())

odict_keys(['pe'])


In [10]:
print([p for p in pe.parameters()])

[]


## 3. einsum  
[공식 문서 링크](https://pytorch.org/docs/stable/generated/torch.einsum.html)  
[참고 블로그](https://ita9naiwa.github.io/numeric%20calculation/2018/11/10/Einsum.html)

Pytorch, Tensorflow, Numpy 등에 있는 다양한 함수/메서드를 다 외우거나 일일히 찾아보지 않아도 여러 계산들을 편리하게 해주는 도구이다. `einsum` 연산은 Einstein Summation Convention에 따라 연산을 진행하는 방식이라고 한다.  

In [2]:
import numpy as np

A = np.array([[1,2,3], [4,5,6]])
print(A)

[[1 2 3]
 [4 5 6]]


In [3]:
# Transpose
R = np.einsum("ij->ji", A)
print(R)

[[1 4]
 [2 5]
 [3 6]]


In [11]:
# Diagonal
A = np.ones((5, 5))
diag = np.einsum("ii->i", A)
print(diag)

[1. 1. 1. 1. 1.]


In [12]:
# Trace
print(np.einsum("ii->", A))

5.0


In [16]:
# Summation
A = np.array([[1,2,3], [4,5,6]])
R = np.einsum("ij->", A)
print(A)
print(R)

row_sum = np.einsum("ij->i", A)
col_sum = np.einsum("ij->j", A)
print(row_sum, col_sum)

[[1 2 3]
 [4 5 6]]
21
[ 6 15] [5 7 9]


In [19]:
A = np.array(list(range(0, 12))).reshape((3, 2, 2))
print(A)

# jk -> k가 됨
print(np.einsum("ijk->ik", A))

[[[ 0  1]
  [ 2  3]]

 [[ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]]]
[[ 2  4]
 [10 12]
 [18 20]]


In [27]:
# multiplication
import torch

# outer product
x = torch.Tensor([1, 2, 3, 4])
y = torch.Tensor([1, 1, 1, 1])
print(torch.einsum("i,j->ij", x, y))

# inner product
print(torch.einsum("i,j->", x, y))

# element wise multiplication
print(torch.einsum("i,i->i", x, y))

# matrix-vector multiplication
c = torch.Tensor([[2,2,2,2], [3,3,3,3]])
print(torch.einsum("ij,j->i", c, y))

tensor([[1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [3., 3., 3., 3.],
        [4., 4., 4., 4.]])
tensor(40.)
tensor([1., 2., 3., 4.])
tensor([ 8., 12.])


In [29]:
# Batch matrix multiplication
# b=3, i=2, j=5, k=4
# b -> batch size, j -> 공통 dimension
# 결과는 (3, 2, 4)가 되어야겠지?
As = torch.randn(3,2,5)
Bs = torch.randn(3,5,4)

result = torch.einsum("bij,bjk->bik", As, Bs)
print(result.shape)

torch.Size([3, 2, 4])


## 4. next